In [2]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import PIL
import pytesseract
import os
import re
import cv2
from io import BytesIO
import easyocr
import io
import pyocr
import openpyxl

In [3]:
#эта первый ищем якорь на картинке
#dzen - якорь - день/дней 
#телега - якорь - err
#vk - якорь подписчик (выше приоритет) - участник (ниже приоритет) 
#youtube - якоря: подписчики + время просмотра

#Юра допиши тут регулярки для поиска
tg = 'err'
vk = ['подпис', 'участ']
youtube_sub = 'подпис'
youtube_time = 'просмотра'

#тут нужно добавить параметр функции выбор соцсети
#надо ещё доработать эту функцию, мне кажется за счёт
#улучшения качества фото можно улучшить предикты модели
def find_word_position(img_path):
    image = Image.open(img_path)
    result = image.convert('L')
    image = result.filter(ImageFilter.SHARPEN)
    gray = np.asarray(image)
    result = reader.readtext(gray, batch_size=512, min_size=5)
    #заменить на выбор соцсети
    word_to_find = 'ень'
    word_to_find2 = 'ней'
    #тут один возврат, ютуб в 2 захода
    for r in result:
        text = r[1]
        if word_to_find in text:
            return np.asarray(r[0][0])
        elif word_to_find2 in text:
            return np.asarray(r[0][0])

In [6]:
reader = easyocr.Reader(['ru'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [7]:
#Сюда будет приходить путь до временного файла в папке на сервере
#файл загружается через интерфейс сайта

#Функция обработки изображения
def process_image(path:str):
    #открываем изображение с помощью Pillow
    image = Image.open(path)
    #отправляем изображение искать якорь
    top_pos = find_word_position(path)
    #если есть результат - обрезаем квадрат

    #кстати можно улучшить модель, если правильно определять размер
    #до того как отравлять её на поиск - на мобилках и компах разное 
    #соотношение сторон
    width, height = image.size
    if type(top_pos)==np.ndarray:
        load = 1
        cropped_image = image.crop((top_pos[0]-100, top_pos[1], width, top_pos[1]+height/10))
    #если нет - пытаемся обрезать наугад
    else:
        load = 0
        #тут нужно резать в зависимости от соцсети и соотношения сторон или лучше вообще не резать - думаю
        cropped_image = image.crop((width, height*0.6, width, height)
    #переводим в серый
    result = cropped_image.convert('L')
    image = image.filter(ImageFilter.SHARPEN) # Увеличение резкости   
    #image.show()
    #превращаем в массив numpy
    gray = np.asarray(image)
    #читаем с помощью easyocr
    result = reader.readtext(gray, batch_size=512, detail=0, min_size=3)
    return result, load#, gray_video

In [8]:
text = pd.DataFrame(columns=['файл', 'текст', 'Успешная загрузка', 'Дочитывания'])

In [9]:
# Путь к папке, в которой находятся папки с изображениями
root_folder = "temp"

In [10]:
# Обход всех папок и файлов внутри главной папки
for root, dirs, files in os.walk(root_folder):
    for file in files:
    # Проверка, что файл - изображение (замените на поддерживаемые форматы по своему усмотрению)
        if file.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')):
            # Получение полного пути к файлу
            file_path = os.path.join(root, file)
            # Обработка изображения с помощью функции process_image
            string,load  = process_image(file_path)
            if(load==1):
                print(str(file)+'\t'+'Успешно загружен')
                text.loc[len(text)] = [file, string, 'Успешно загружен', '-']
            elif(load==0):
                print(str(file)+'\t'+'Требуется ручная проверка')
                text.loc[len(text)] = [file, string, 'Требуется ручная проверка', '-']



017619a5-e33a-4351-a695-0d04da4519b8.png	Успешно загружен
04c3357a-3cee-43a1-824b-b1151e6d6404.jpg	Успешно загружен
0578d9d6-0353-4a1c-9fad-91e1f2d9864e.png	Успешно загружен
06173a6f-a7a6-4313-af95-5affed21df31.png	Требуется ручная проверка
0967ba80-8a20-4a30-94e5-3a52b0a451b4.jpg	Успешно загружен
0cdea12d-b73c-40cb-a5ee-3ae985ce862d.png	Успешно загружен
0f00d72f-cf11-45d2-8051-2611bcbde777.png	Успешно загружен
16066bfa-3ce1-4a54-a944-5beb0bae6f1b.png	Успешно загружен
1a114fb6-9546-4087-9337-0bc5f1921baa.png	Успешно загружен
1da9aa5e-ba13-411d-a6b8-bd18a24949b4.png	Успешно загружен
2177cd4e-441d-4aaa-84f8-2557c71a6aa5.jpg	Успешно загружен
23fc51b7-3a14-41be-abb2-d180ce7937b7.JPG	Успешно загружен
2cda9a39-9c83-4e3e-ac79-6b0ad0358a42.jpg	Успешно загружен
3118e8c9-8682-4b08-99fd-911b682b61a1.jpg	Требуется ручная проверка
3279f8d9-fe88-44a0-8e88-9ef789c2d567.png	Успешно загружен
3389deb0-973f-4aba-b5a0-61dac558eda9.JPG	Успешно загружен
3422262e-677c-465f-9b4d-aa8e5dc6dd89.png	Успешно загру

AttributeError: 'DataFrame' object has no attribute 'excel'

In [11]:
text.to_excel('dzen.xlsx')

In [12]:
df = pd.read_excel('dzen.xlsx')

In [16]:
df.loc[2]['текст']

"['Дзен-студия', 'Наммиен шобывыймувполноэкранно режима', 'Главное', 'Статистика', '^]', 'Статистика', 'Подписчики', 'Публикации', 'Почасовая статистика', 'Доход', 'Аудитория', '01 _ 31 мая 2023', 'Публикации', 'Все', 'Статьи', 'Посты', 'Видео', 'Только подписчики', 'По дате события', 'По дате публикации', 'Отчёт', 'Комментарии', '₽', 'Монетизация', 'Дочитывания и просмотры', 'Детализация', 'по дням', 'Настройки', '202', 'Всё', 'Дзене', 'Справка', 'Поддержка', '1??', '1 мая', '07 мая', '19 иая', 'кав', '31 мая', 'ЗаГолсЕак', 'Дата', 'Ок=з=', 'Дочитыв:ния', 'Обшее время', 'ПодГиски', 'проснотры', 'прасматра', 'Всего за 31 день', '33 011', '1 280', '15.5 ч:', '52', '12', '280', '37 публикаций', '0', 'Ваш канал', '26 мая', 'Неосновательное обогащение:', 'развенчиваем мифы', '293', '19', '10', '~п7-']"